In [1]:
# --------------------------------------------------------------------------
# 1. 라이브러리 임포트 및 기본 설정
# --------------------------------------------------------------------------
import os
import yaml
import pandas as pd
import seaborn as sns
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np
from ultralytics.utils.torch_utils import model_info

# --------------------------------------------------------------------------
# 2. 실험 환경 및 하이퍼파라미터 설정
# --------------------------------------------------------------------------

# 실험 대상 모델 리스트
MODELS_TO_TEST = ['yolov5n', 'yolov8n', 'yolov10n'] # 예시로 모델 수를 줄임, 필요시 수정
# 데이터 증강 옵션 리스트
AUGMENT_OPTIONS = [True, False]

# 데이터 및 모델 저장 경로 (사용자 환경에 맞게 수정)
DATA_PATH = r"D:\Virtual_Fahion\GraduationThesis2\data"
MODEL_SAVE_DIR = r"D:\Virtual_Fahion\GraduationThesis2\saveModels"

# 하이퍼파라미터
EPOCHS = 30
BATCH_SIZE = 16
IMG_SIZE = 640
OPTIMIZER = 'Adam'

# GPU 설정
DEVICE = '0' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

# 데이터 YAML 파일 로드
data_yaml_path = os.path.join(DATA_PATH, 'data.yaml')
with open(data_yaml_path) as f:
    data_config = yaml.safe_load(f)
print(f"Dataset classes: {data_config['names']}")

# F1-Score 계산 함수
def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall + 1e-9)

Using device: 0
Dataset classes: ['angry', 'contempt', 'disgust', 'fear', 'happy', 'natural', 'sad', 'sleepy', 'surprised']


In [3]:
# --------------------------------------------------------------------------
# 3. 모델별, 증강 옵션별 학습, 검증 및 결과 분석
# --------------------------------------------------------------------------
all_results_summary = []

for model_name in MODELS_TO_TEST:
    for augment_status in AUGMENT_OPTIONS:
        
        run_name = f'train_{model_name}_aug_{augment_status}'
        display_name = f'{model_name} (Aug: {augment_status})'
        
        print("\n" + "="*60)
        print(f"🚀 STARTING EXPERIMENT FOR MODEL: {display_name}")
        print("="*60 + "\n")

        # 1. 모델 학습
        model = YOLO(f"{model_name}.pt")
        model.train(
            data=data_yaml_path, epochs=EPOCHS, batch=BATCH_SIZE, imgsz=IMG_SIZE,
            device=DEVICE, project=MODEL_SAVE_DIR, name=run_name,
            exist_ok=True, save=True, pretrained=True, 
            augment=augment_status, plots=True, optimizer=OPTIMIZER
        )

        # 2. 가장 성능 좋은 모델 로드
        best_model_path = os.path.join(MODEL_SAVE_DIR, run_name, 'weights', 'best.pt')
        best_model = YOLO(best_model_path)

        # 3. 모델 성능 검증
        val_results = best_model.val(
            data=data_yaml_path, split='val', batch=BATCH_SIZE,
            imgsz=IMG_SIZE, device=DEVICE, plots=True
        )
        
        # ★★★★★ [ 신규 추가된 부분 ] ★★★★★
        # 4. 학습 과정 및 클래스별 성능 시각화
        print("\n" + "-"*50)
        print(f"📈 VISUALIZING TRAINING & VALIDATION ANALYSIS FOR: {display_name}")
        print("-"*50)
        
        # 2개의 그래프를 나란히 그릴 도화지 준비
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))
        fig.suptitle(f'Analysis for {display_name}', fontsize=16, weight='bold')

        # 4-1. 학습 과정 그래프 (results.png) 불러오기
        results_png_path = os.path.join(MODEL_SAVE_DIR, run_name, 'results.png')
        if os.path.exists(results_png_path):
            img_results = cv2.imread(results_png_path)
            axes[0].imshow(cv2.cvtColor(img_results, cv2.COLOR_BGR2RGB))
            axes[0].set_title('Training & Validation Curves')
            axes[0].axis('off')
        else:
            axes[0].text(0.5, 0.5, 'results.png not found', ha='center'); axes[0].axis('off')

        # 4-2. 클래스별 혼동 행렬 (confusion_matrix.png) 불러오기
        cm_png_path = os.path.join(MODEL_SAVE_DIR, run_name, 'confusion_matrix.png')
        if os.path.exists(cm_png_path):
            img_cm = cv2.imread(cm_png_path)
            axes[1].imshow(cv2.cvtColor(img_cm, cv2.COLOR_BGR2RGB))
            axes[1].set_title('Confusion Matrix')
            axes[1].axis('off')
        else:
            axes[1].text(0.5, 0.5, 'confusion_matrix.png not found', ha='center'); axes[1].axis('off')
            
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()        

        # 5. 최종 평가 지표 추출
        print("\n" + "-"*50)
        print(f"📊 EXTRACTING PERFORMANCE INDICATORS FOR: {display_name}")
        print("-"*50)
        
        map50 = val_results.box.map50
        precision = val_results.box.p.mean()
        recall = val_results.box.r.mean()
        
        speed = val_results.speed
        latency_ms = speed['preprocess'] + speed['inference'] + speed['postprocess']
        
        info = model_info(best_model.model, detailed=True, imgsz=IMG_SIZE)
        params_m = info[1] / 1e6 
        gflops = info[3]

        # 6. 통합 결과 리스트에 추가
        all_results_summary.append({
            "Model": display_name, "mAP@50": map50, "Precision": precision,
            "Recall": recall, "Latency (ms)": latency_ms,
            "Parameters (M)": params_m, "GFLOPs": gflops
        })


🚀 STARTING EXPERIMENT FOR MODEL: yolov5n (Aug: True)

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

New https://pypi.org/project/ultralytics/8.3.203 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.198  Python-3.9.23 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 49140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Virtual_Fahion\GraduationThesis2\data\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=

<Figure size 2000x800 with 2 Axes>


--------------------------------------------------
📊 EXTRACTING PERFORMANCE INDICATORS FOR: yolov5n (Aug: True)
--------------------------------------------------
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False        1728       [16, 3, 6, 6]  -0.00934      2.05        float32
    0                       model.0.conv.bias              Conv2d     False          16                [16]      0.49      5.28        float32
    1                             model.0.act                SiLU     False           0                  []         -         -              -
    2                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]  -0.00358     0.193        float32
    2                       model.1.conv.bias              Conv2d     False          32                [32]     0.882      6.37        f

<Figure size 2000x800 with 2 Axes>


--------------------------------------------------
📊 EXTRACTING PERFORMANCE INDICATORS FOR: yolov5n (Aug: False)
--------------------------------------------------
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False        1728       [16, 3, 6, 6]  -0.00934      2.05        float32
    0                       model.0.conv.bias              Conv2d     False          16                [16]      0.49      5.28        float32
    1                             model.0.act                SiLU     False           0                  []         -         -              -
    2                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]  -0.00358     0.193        float32
    2                       model.1.conv.bias              Conv2d     False          32                [32]     0.882      6.37        

...


--------------------------------------------------
📊 EXTRACTING PERFORMANCE INDICATORS FOR: yolov8n (Aug: False)
--------------------------------------------------
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False         432       [16, 3, 3, 3]   -0.0446      5.18        float32
    0                       model.0.conv.bias              Conv2d     False          16                [16]      1.42      6.68        float32
    1                             model.0.act                SiLU     False           0                  []         -         -              -
    2                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]  0.000422     0.242        float32
    2                       model.1.conv.bias              Conv2d     False          32                [32]     -1.03      5.94        

<Figure size 2000x800 with 2 Axes>


--------------------------------------------------
📊 EXTRACTING PERFORMANCE INDICATORS FOR: yolov10n (Aug: True)
--------------------------------------------------
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False         432       [16, 3, 3, 3]   -0.0638      4.83        float32
    0                       model.0.conv.bias              Conv2d     False          16                [16]     0.991       7.3        float32
    1                             model.0.act                SiLU     False           0                  []         -         -              -
    2                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]   0.00096     0.136        float32
    2                       model.1.conv.bias              Conv2d     False          32                [32]     -5.03       7.2        

<Figure size 2000x800 with 2 Axes>


--------------------------------------------------
📊 EXTRACTING PERFORMANCE INDICATORS FOR: yolov10n (Aug: False)
--------------------------------------------------
layer                                    name                type  gradient  parameters               shape        mu     sigma
    0                     model.0.conv.weight              Conv2d     False         432       [16, 3, 3, 3]   -0.0526      4.75        float32
    0                       model.0.conv.bias              Conv2d     False          16                [16]     0.853      7.14        float32
    1                             model.0.act                SiLU     False           0                  []         -         -              -
    2                     model.1.conv.weight              Conv2d     False        4608      [32, 16, 3, 3]  0.000556     0.138        float32
    2                       model.1.conv.bias              Conv2d     False          32                [32]     -5.01      6.38       

In [4]:
# --------------------------------------------------------------------------
# 4. 모든 모델 실험 결과 통합 비교 분석
# --------------------------------------------------------------------------
print("\n" + "="*60)
print("🏆 FINAL RESULTS SUMMARY FOR ALL MODELS 🏆")
print("="*60 + "\n")

summary_df = pd.DataFrame(all_results_summary).set_index("Model")
print(summary_df.sort_values(by="mAP@50", ascending=False).to_string(formatters={
    'mAP@50': '{:.3f}'.format, 'Precision': '{:.3f}'.format, 'Recall': '{:.3f}'.format,
    'Latency (ms)': '{:.2f}'.format, 'Parameters (M)': '{:.2f}'.format, 'GFLOPs': '{:.2f}'.format
}))


🏆 FINAL RESULTS SUMMARY FOR ALL MODELS 🏆

                      mAP@50 Precision Recall Latency (ms) Parameters (M) GFLOPs
Model                                                                           
yolov8n (Aug: True)    0.712     0.668  0.636         2.00           3.01   8.09
yolov8n (Aug: False)   0.709     0.677  0.619         2.04           3.01   8.09
yolov5n (Aug: True)    0.683     0.650  0.621         2.17           2.50   7.07
yolov5n (Aug: False)   0.683     0.650  0.621         2.02           2.50   7.07
yolov10n (Aug: False)  0.654     0.592  0.611         1.15           2.27   6.54
yolov10n (Aug: True)   0.650     0.587  0.604         1.38           2.27   6.54


In [5]:
# --------------------------------------------------------------------------
# 5. 성능 지표 시각화
# --------------------------------------------------------------------------
print("\n\n" + "="*60)
print("📈 VISUALIZING PERFORMANCE COMPARISON 📈")
print("="*60 + "\n")

plt.style.use('seaborn-v0_8-whitegrid')

# 1. 정확도/속도 트레이드오프 시각화 (Scatter Plot)
fig, ax = plt.subplots(figsize=(12, 8))
sns.scatterplot(
    data=summary_df, x="Latency (ms)", y="mAP@50",
    hue="Model", size="Parameters (M)", sizes=(100, 1000),
    alpha=0.8, palette="viridis", ax=ax
)
ax.set_title("Accuracy vs. Latency Trade-off", fontsize=18, weight='bold')
ax.set_xlabel("Latency (ms) - Lower is Faster", fontsize=12)
ax.set_ylabel("mAP@50 - Higher is Better", fontsize=12)
ax.legend(title="Model", bbox_to_anchor=(1.05, 1), loc='upper left')
for i, row in summary_df.iterrows():
    ax.text(row['Latency (ms)']+0.1, row['mAP@50'], i, fontsize=9, ha='left')
plt.grid(True)
plt.tight_layout()

# ★★★★★ [ 신규 추가된 부분 ] ★★★★★
# 현재 그림을 파일로 저장 (plt.show() 이전에 호출)
plt.savefig('tradeoff_plot.png', dpi=300, bbox_inches='tight')
print("\nAccuracy vs. Latency tradeoff plot saved to 'tradeoff_plot.png'")
# ★★★★★ [ 여기까지 신규 추가된 부분 ] ★★★★★

plt.show()

# 2. 개별 성능 지표 비교 (Bar Plots)
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('Individual Performance Indicator Comparison', fontsize=20, weight='bold')
metrics_to_plot = ["mAP@50", "Latency (ms)", "Parameters (M)", "GFLOPs"]
for ax, metric in zip(axes.flatten(), metrics_to_plot):
    ascending = True if "Latency" in metric or "Parameters" in metric or "GFLOPs" in metric else False
    sorted_df = summary_df.sort_values(by=metric, ascending=ascending)
    colors = plt.cm.plasma(np.linspace(0, 1, len(sorted_df)))
    bars = ax.barh(sorted_df.index, sorted_df[metric], color=colors)
    ax.set_title(metric, fontsize=14)
    ax.set_xlabel("Value", fontsize=12)
    for bar in bars:
        width = bar.get_width()
        ax.text(width * 1.01, bar.get_y() + bar.get_height()/2., f'{width:.2f}',
                va='center', ha='left', fontsize=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# ★★★★★ [ 신규 추가된 부분 ] ★★★★★
# 현재 그림을 파일로 저장 (plt.show() 이전에 호출)
plt.savefig('comparison_plots.png', dpi=300)
print("Individual performance comparison plots saved to 'comparison_plots.png'")
# ★★★★★ [ 여기까지 신규 추가된 부분 ] ★★★★★

plt.show()

print("\n\n🎉 All experiments completed.")



📈 VISUALIZING PERFORMANCE COMPARISON 📈


Accuracy vs. Latency tradeoff plot saved to 'tradeoff_plot.png'


<Figure size 1200x800 with 1 Axes>

Individual performance comparison plots saved to 'comparison_plots.png'


<Figure size 1800x1400 with 4 Axes>



🎉 All experiments completed.
